In [18]:
import pandas as pd

from features_extract import bert_tokenizer, my_own_tokenizer, clean_tokens
from features_extract import filter_tokens_by_freq, create_frequency_dict

In [19]:
data_file = r".\\chaverim_makshivim-train.xlsx"
data = pd.read_excel(data_file)

In [20]:
useless_cols = ['Seq', 'Site', 'URL', 'ID', 'Hdate', 'Rabby', 'Subject', 'Tags', 'SubjectLen', 'Year',
       'Hyear', 'Hmonth', 'Hday', '1 גברים', '2 נשים']
data = data.drop(useless_cols, axis=1)

In [21]:
data.head(2)

,Question,Answer,QuestionLen,AnswerLen,Gender,AnswerGender
0,"\nשלום לכם, אני נער בן 14 וגר באחת ה""התנחלויות...","\nב""ה\nד´ שלום!\nראשית רציתי לומר לך שאני מאוד...",182,1257,1,1.0
1,\nלעיתים יוצא לנו להתחיל את הסעודה בראש חודש ו...,"\nב""ה\nשלום וברכה\nהדבר נתון במחלוקת הפוסקים ה...",286,1095,1,1.0


## Tokenization

Two different methods were used in order to tokenize the data:
* Simple tokenizer - by spliting each text by spaces, with some preprocessing
* Bert tokenizer - implementation created with the help of Shlomi Batito :),
    taken from - https://huggingface.co/onlplab/alephbert-base

In [22]:
def create_tokens_columns(df, sw = True):
    df["Q_tokens_no_sw"] = df.apply(lambda r: my_own_tokenizer(r['Question']), axis=1).apply(
        lambda x: clean_tokens(x, stop_words=sw)
    )

    df["A_tokens_no_sw"] = df.apply(lambda r: my_own_tokenizer(r['Answer']), axis=1).apply(
        lambda x: clean_tokens(x, stop_words=sw)
    )

def create_bert_tokens_columns(df, sw = True):
    df["Q_bert_tokens_no_sw"] = df.apply(lambda r: bert_tokenizer(r['Question']), axis=1).apply(
        lambda x: clean_tokens(x, stop_words=sw)
    )

    df["A_bert_tokens_no_sw"] = df.apply(lambda r: bert_tokenizer(r['Answer']), axis=1).apply(
        lambda x: clean_tokens(x, stop_words=sw)
    )

In [23]:
create_tokens_columns(data)

In [24]:
data.head(3)

,Question,Answer,QuestionLen,AnswerLen,Gender,AnswerGender,Q_tokens_no_sw,A_tokens_no_sw
0,"\nשלום לכם, אני נער בן 14 וגר באחת ה""התנחלויות...","\nב""ה\nד´ שלום!\nראשית רציתי לומר לך שאני מאוד...",182,1257,1,1.0,"[שלום, נער, בן, וגר, באחת, התנחלויות, אשמח, לד...","[שלום, ראשית, רציתי, לומר, לך, שאני, מבין, תחו..."
1,\nלעיתים יוצא לנו להתחיל את הסעודה בראש חודש ו...,"\nב""ה\nשלום וברכה\nהדבר נתון במחלוקת הפוסקים ה...",286,1095,1,1.0,"[לעיתים, יוצא, להתחיל, הסעודה, בראש, חודש, ולס...","[שלום, וברכה, הדבר, נתון, במחלוקת, הפוסקים, הא..."
2,\nא ישר כח על האתר!\nאיך אפשר לנצל את החופש בצ...,"\nלק""י\n\nשלום לך \nראשית, תודה על המילים החמו...",198,767,1,1.0,"[ישר, כח, האתר, אפשר, לנצל, החופש, בצורה, נורמ...","[לק, שלום, לך, ראשית, תודה, המילים, החמות, באמ..."


In [25]:
create_bert_tokens_columns(data)

In [26]:
[print(i) for i in data[["Q_tokens_no_sw", "Q_bert_tokens_no_sw", "A_tokens_no_sw", "A_bert_tokens_no_sw"]].iloc[44]]

['בעצם', 'ממש', 'רוצה', 'לחזור', 'בתשובה', 'מאמין', 'באלוקים', 'לצערי', 'מצליח', 'להפוך', 'האמונה', 'הזאת', 'למשהו', 'מוחשי', 'היראת', 'שמיים', 'נגמרת', 'כמעט', 'מייד', 'כשמתחיל', 'הנסיון', 'ואני', 'נופל', 'הרבה', 'פעמים', 'ושוב', 'בשל', 'בכלל', 'אדם', 'דתי', 'חטאתי', 'במזיד', 'ואף', 'פי', 'שהרגשתי', 'במודע', 'המשכתי', 'והמשכתי', 'ובכיתי', 'ועשיתי', 'תשובה', 'ונפלתי', 'ושוב', 'אפילו', 'מהשמיים', 'קיבלתי', 'רמז', 'ברור', 'בצורת', 'חלום', 'שלא', 'הותיר', 'לספק', 'יודע', 'ומרגיש', 'שאני', 'מבזבז', 'הזמן', 'היקר', 'שהקב', 'הקציב', 'בעולם', 'הזה', 'ואפילו', 'צביעות', 'עדיף', 'בכלל', 'לנסות', 'לחזור', 'בתשובה']
['בעצם', 'ממש', 'רוצה', 'לחזור', 'בתשובה', 'מאמין', 'באלו', '##קים', 'לצערי', 'מצליח', 'להפוך', 'האמונה', 'הזאת', 'למשהו', 'מוחשי', 'היר', '##את', 'שמיים', 'נגמרת', 'כמעט', 'מייד', 'כשמת', '##חיל', 'הנסיון', 'ואני', 'נופל', 'הרבה', 'פעמים', 'ושוב', 'בשל', 'בכלל', 'אדם', 'דתי', 'חטאתי', 'במז', '##יד', 'ואף', 'פי', 'שהרגשתי', 'במודע', 'המשכתי', 'והמש', '##כתי', 'ובכי', '##תי', 'ועשיתי',

[None, None, None, None]

In [27]:
data.head(3)

,Question,Answer,QuestionLen,AnswerLen,Gender,AnswerGender,Q_tokens_no_sw,A_tokens_no_sw,Q_bert_tokens_no_sw,A_bert_tokens_no_sw
0,"\nשלום לכם, אני נער בן 14 וגר באחת ה""התנחלויות...","\nב""ה\nד´ שלום!\nראשית רציתי לומר לך שאני מאוד...",182,1257,1,1.0,"[שלום, נער, בן, וגר, באחת, התנחלויות, אשמח, לד...","[שלום, ראשית, רציתי, לומר, לך, שאני, מבין, תחו...","[שלום, נער, בן, וגר, באחת, התנחלויות, אשמח, לד...","[שלום, ראשית, רציתי, לומר, לך, שאני, מבין, תחו..."
1,\nלעיתים יוצא לנו להתחיל את הסעודה בראש חודש ו...,"\nב""ה\nשלום וברכה\nהדבר נתון במחלוקת הפוסקים ה...",286,1095,1,1.0,"[לעיתים, יוצא, להתחיל, הסעודה, בראש, חודש, ולס...","[שלום, וברכה, הדבר, נתון, במחלוקת, הפוסקים, הא...","[לעיתים, יוצא, להתחיל, הסעודה, בראש, חודש, ולס...","[שלום, וברכה, הדבר, נתון, במחלוקת, הפוסקים, הא..."
2,\nא ישר כח על האתר!\nאיך אפשר לנצל את החופש בצ...,"\nלק""י\n\nשלום לך \nראשית, תודה על המילים החמו...",198,767,1,1.0,"[ישר, כח, האתר, אפשר, לנצל, החופש, בצורה, נורמ...","[לק, שלום, לך, ראשית, תודה, המילים, החמות, באמ...","[ישר, כח, האתר, אפשר, לנצל, החופש, בצורה, נורמ...","[לק, שלום, לך, ראשית, תודה, המילים, החמות, באמ..."


## Vectorization

Two different methods were used for vectorization:
* Count Vectorization
* TFIDF Vectorization

Plus, i tried some Hand Crafted features, that are interesting to examine.

In [28]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Inorder to save runtime and memory (had a lot of OutOfMemory exceptions), 
    i fitltered out tokens that had low TF across the corpus.

For comparison - vocab sizes before such filtering:
* vect_q - Size of vocabular is: 45223
* vect_a - Size of vocabular is: 109157
* vect_q_bert - Size of vocabular is: 27112
* vect_a_bert - Size of vocabular is: 36539

In [29]:
def create_filter_vocab_by_freq(tokens_corpus, min_freq):
    filtered_tokens_corpus = filter_tokens_by_freq(tokens_corpus, min_freq)
    filtered_vocab = {item for sublist in filtered_tokens_corpus for item in sublist}

    return filtered_vocab

In [30]:
q_vocab = create_filter_vocab_by_freq(data["Q_tokens_no_sw"], 50)
a_vocab = create_filter_vocab_by_freq(data["A_tokens_no_sw"], 50)
q_bert_vocab = create_filter_vocab_by_freq(data["Q_bert_tokens_no_sw"], 50)
a_bert_vocab = create_filter_vocab_by_freq(data["A_bert_tokens_no_sw"], 50)

In [31]:
print(len(q_vocab))
print(len(a_vocab))
print(len(q_bert_vocab))
print(len(a_bert_vocab))

1159
5062
1422
7018


In [32]:
d = create_frequency_dict(data["Q_tokens_no_sw"])
print(d["אֲדוֹנֵנוּ"])

d2 = create_frequency_dict(data["A_bert_tokens_no_sw"])
print(d2["##אולה"])

##אולה

22
111


### Count Vectorizer

In [33]:
# CountVectorizer useally takes corpus of docs as parameter
#   Since we already tokenized the data, we need to make a little change

# We assumed that tokens_corpus is of the form:
# tokens_corpus = [['This', 'is', 'doc1'], ['And', 'that', 'is', 'doc2'],['doc3'], ...]

def dummy(doc):
    return doc

def create_count_vector_df(tokens_corpus, vocab):
    cv = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
        vocabulary=vocab
    )

    count_vector = cv.fit_transform(tokens_corpus)
    print("Size of vocabular is:", len(cv.vocabulary_.keys()))
    
    count_vect_df = pd.DataFrame(count_vector.todense(), columns=cv.get_feature_names_out())
    
    return count_vect_df

In [34]:
count_vect_q_df = create_count_vector_df(data["Q_tokens_no_sw"], q_vocab)
count_vect_a_df = create_count_vector_df(data["A_tokens_no_sw"], a_vocab)
count_vect_q_bert_df = create_count_vector_df(data["Q_bert_tokens_no_sw"], q_bert_vocab)
count_vect_a_bert_df = create_count_vector_df(data["A_bert_tokens_no_sw"], a_bert_vocab)

c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Size of vocabular is: 1159
Size of vocabular is: 5062


c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Size of vocabular is: 1422


c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Size of vocabular is: 7018


Create Data Frames of the features, and save as csv files:

In [35]:
count_vect_q_df.head(3)

,אבא,אבי,אגב,אדם,אהבה,אהיה,אודה,אוהב,אוהבים,אוהבת,...,תלוי,תמיד,תעזרו,תענו,תפילה,תפילות,תפילין,תקופה,תשובה,תשובות
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
count_vect_a_df.head(3)

,אֲשֶׁר,אֶת,אב,אבא,אבות,אבותינו,אבי,אביו,אביך,אבינו,...,תשמח,תשעה,תשתדלי,תתחיל,תתחילי,תתייאש,תתן,תתני,תתפלל,תתפללי
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
count_vect_q_bert_df.head(3)

,##א,##אה,##אות,##אלה,##אש,##אשת,##את,##ב,##בה,##בו,...,תמיד,תעזרו,תענו,תפילה,תפילות,תפילין,תפילת,תקופה,תשובה,תשובות
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
count_vect_a_bert_df.head(3)

,##s,##א,##אב,##אג,##אד,##אה,##או,##אול,##אולה,##אור,...,תתחיל,תתחילי,תתי,תתן,תתני,תתע,תתפ,תתק,תתקבל,תתר
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
def concat_feat_with_data(data, feat_df):
    col_from_data = ["Question", "Answer", "QuestionLen", "AnswerLen", "Gender"]

    data_with_feat = pd.concat([data[col_from_data], feat_df], axis=1)

    return data_with_feat

In [40]:
def write_data_with_feat_to_csv(df, folder, file_name):
    df.to_csv(folder + file_name)

In [41]:
featurs_folder = r"C:\\Users\\elroi\\OneDrive\\Desktop\\DataMining\\InformationRetrieval\\project\\features\\"

features = [count_vect_q_df, count_vect_a_df, count_vect_q_bert_df, count_vect_a_bert_df]
names = ["count_vector_above50_no_sw_Q.csv", "count_vector_above50_no_sw_A.csv", 
         "count_vector_above50_no_sw_Q_bert.csv", "count_vector_above50_no_sw_A_bert.csv"]

for feat, name in zip(features, names):
    data_with_feat = concat_feat_with_data(data, feat)
    write_data_with_feat_to_csv(data_with_feat, featurs_folder, name)

### TFIDF Vectorizer

In [42]:
def dummy(doc):
    return doc

def create_tfidf_vector_df(tokens_corpus, vocab):
    tfidf = TfidfVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
        vocabulary=vocab
    )
    
    tfidf_vector = tfidf.fit_transform(tokens_corpus)
    
    tfidf_vect_df = pd.DataFrame(tfidf_vector.todense(), columns=tfidf.get_feature_names_out())
    
    return tfidf_vect_df

In [43]:
tfidf_vect_q_df = create_tfidf_vector_df(data["Q_tokens_no_sw"], q_vocab)
tfidf_vect_a_df = create_tfidf_vector_df(data["A_tokens_no_sw"], a_vocab)
tfidf_vect_q_bert_df = create_tfidf_vector_df(data["Q_bert_tokens_no_sw"], q_bert_vocab)
tfidf_vect_a_bert_df = create_tfidf_vector_df(data["A_bert_tokens_no_sw"], a_bert_vocab)

c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [44]:
print(tfidf_vect_q_df.head(2))
print(tfidf_vect_a_df.head(2))
print(tfidf_vect_q_bert_df.head(2))
print(tfidf_vect_a_bert_df.head(2))

   אבא  אבי  אגב  אדם  אהבה  אהיה  אודה  אוהב  אוהבים  אוהבת  ...  תלוי  תמיד  \
0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0     0.0    0.0  ...   0.0   0.0   
1  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0     0.0    0.0  ...   0.0   0.0   

   תעזרו  תענו  תפילה  תפילות  תפילין  תקופה  תשובה  תשובות  
0    0.0   0.0    0.0     0.0     0.0    0.0    0.0     0.0  
1    0.0   0.0    0.0     0.0     0.0    0.0    0.0     0.0  

[2 rows x 1159 columns]
   אֲשֶׁר  אֶת   אב  אבא  אבות  אבותינו  אבי  אביו  אביך  אבינו  ...  תשמח  \
0     0.0  0.0  0.0  0.0   0.0      0.0  0.0   0.0   0.0    0.0  ...   0.0   
1     0.0  0.0  0.0  0.0   0.0      0.0  0.0   0.0   0.0    0.0  ...   0.0   

   תשעה  תשתדלי  תתחיל  תתחילי  תתייאש  תתן  תתני  תתפלל  תתפללי  
0   0.0     0.0    0.0     0.0     0.0  0.0   0.0    0.0     0.0  
1   0.0     0.0    0.0     0.0     0.0  0.0   0.0    0.0     0.0  

[2 rows x 5062 columns]
   ##א  ##אה  ##אות  ##אלה  ##אש  ##אשת  ##את  ##ב  ##בה  ##בו  ...  תמיד  \
0  0.0 

In [45]:
print(1159 + 5062)
print(7018 + 1422)

6221
8440


In [46]:
def concat_tfidf_vectors(df1, df2):
    return pd.concat([df1, df2.rename(columns=lambda x: "Answer_" + x)], axis=1)

In [47]:
tfidf_vect_q_and_a_df = concat_tfidf_vectors(tfidf_vect_q_df, tfidf_vect_a_df)
tfidf_vect_q_and_a_bert_df = concat_tfidf_vectors(tfidf_vect_q_bert_df, tfidf_vect_a_bert_df)

In [48]:
print(tfidf_vect_q_and_a_df.head(2))
print(tfidf_vect_q_and_a_bert_df.head(2))

   אבא  אבי  אגב  אדם  אהבה  אהיה  אודה  אוהב  אוהבים  אוהבת  ...  \
0  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0     0.0    0.0  ...   
1  0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0     0.0    0.0  ...   

   Answer_תשמח  Answer_תשעה  Answer_תשתדלי  Answer_תתחיל  Answer_תתחילי  \
0          0.0          0.0            0.0           0.0            0.0   
1          0.0          0.0            0.0           0.0            0.0   

   Answer_תתייאש  Answer_תתן  Answer_תתני  Answer_תתפלל  Answer_תתפללי  
0            0.0         0.0          0.0           0.0            0.0  
1            0.0         0.0          0.0           0.0            0.0  

[2 rows x 6221 columns]
   ##א  ##אה  ##אות  ##אלה  ##אש  ##אשת  ##את  ##ב  ##בה  ##בו  ...  \
0  0.0   0.0    0.0    0.0   0.0    0.0   0.0  0.0   0.0   0.0  ...   
1  0.0   0.0    0.0    0.0   0.0    0.0   0.0  0.0   0.0   0.0  ...   

   Answer_תתחיל  Answer_תתחילי  Answer_תתי  Answer_תתן  Answer_תתני  \
0           0.0            0.0       

Number of columns match :)

In [49]:
featurs_folder = r"C:\\Users\\elroi\\OneDrive\\Desktop\\DataMining\\InformationRetrieval\\project\\features\\"

features = [
    tfidf_vect_q_df, tfidf_vect_a_df,
    tfidf_vect_q_bert_df, tfidf_vect_a_bert_df,
    tfidf_vect_q_and_a_df, tfidf_vect_q_and_a_bert_df
]
names = ["tfidf_vector_above50_no_sw_Q.csv", "tfidf_vector_above50_no_sw_A.csv", 
         "tfidf_vector_above50_no_sw_Q_bert.csv", "tfidf_vector_above50_no_sw_A_bert.csv",
         "tfidf_vector_above50_no_sw_Q_and_A.csv", "tfidf_vector_above50_no_sw_Q_and_A_bert.csv"]

for feat, name in zip(features, names):
    data_with_feat = concat_feat_with_data(data, feat)
    write_data_with_feat_to_csv(data_with_feat, featurs_folder, name)

### Verb Male to Female Ratio

List of 100 Most important features of Logistic Regression model run on the question tokens:

In [50]:
top_100_important_features = ['יודעת', 'יודע', 'מרגישה', 'מרגיש', 'בת', 'צריכה', 'חושבת', 'חושב', 'מבינה', 'מצליח', 'מצליחה', 'מאמינה', 'בישיבה', 'לומד', 'אוהבת', 'בן', 'צריך', 'מדריך', 'תורה', 'אומרת', 'מבין', 'אומר', 'אוהב', 'בטוח', 'מאמין', 'לומדת', 'הולכת', 'רב', 'הרב', 'מפחדת', 'חברות', 'מדברת', 'אמורה', 'בנים', 'מדריכה', 'אחת', 'שמחה', 'מדבר', 'מתפללת', 'בטוחה', 'באמת', 'אשמח', 'חבר', 'שואלת', 'שמח', 'הזאת', 'מבקש', 'משתדל', 'שואל', 'המון', 'חייב', 'מפחד', 'וזה', 'מתפלל', 'אנשים', 'הולך', 'תפילין', 'חברים', 'תיכונית', 'אמור', 'ללמוד', 'ישיבה', 'הסדר', 'באולפנה', 'מכיר', 'מצטער', 'להפסיק', 'מתחיל', 'שלא', 'אלא', 'ביום', 'נמצאת', 'חייבת', 'זרע', 'נורא', 'מכירה', 'משתדלת', 'מוצאת', 'דתיה', 'טובה', 'שומרת', 'שאני', 'מצטערת', 'החברות', 'רציתי', 'נופל', 'יעקב', 'חזק', 'מבולבלת', 'צניעות', 'שאלות', 'שומר', 'מלך', 'בעצם', 'מוצא', 'פשוט', 'הברית', 'חברה', 'גמרא', 'תמיד']

Lots of Verbs!

We need to create tokens without removing stop words, for the questions:

In [51]:
data["Q_tokens_sw"] = data.apply(lambda r: my_own_tokenizer(r['Question']), axis=1).apply(
        lambda x: clean_tokens(x, stop_words=False)
    )

In [52]:
# lets remove useless tokens
freq_d = create_frequency_dict(data["Q_tokens_sw"])
# q_full_vocab = create_filter_vocab_by_freq(data["Q_tokens_sw"], 5)

In [53]:
freq_d["אני"]

18038

In [54]:
cv = CountVectorizer(
    ngram_range=(1,2), 
    binary =True, 
    min_df = 500,
    tokenizer=dummy,
    preprocessor=dummy
)

x = cv.fit_transform(data["Q_tokens_sw"])
print("Size of vocabular is:", len(cv.vocabulary_.keys()))
cv_bigrams_df = pd.DataFrame(x.todense(), columns=cv.get_feature_names_out())

c:\Users\elroi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Size of vocabular is: 123


In [55]:
cv_bigrams_df.sum(axis=0).sort_values()

אני מרגישה     504
אותה           509
על זה          509
כזה            512
ואני לא        520
              ... 
זה            3202
את            3231
לי            3476
לא            3886
אני           3999
Length: 123, dtype: int64

In [56]:
male_verbs = ['יודע', 'מרגיש', 'חושב', 'מצליח', 'לומד', 'צריך', 'מדריך', 'מבין', 'אומר', 'אוהב', 'בטוח', 'מאמין', 'מדבר', 'שמח', 'מבקש', 'משתדל', 'שואל', 'חייב', 'מפחד', 'מתפלל', 'הולך', 'אמור', 'מכיר', 'מצטער', 'מתחיל', 'נופל', 'חזק', 'שומר', 'מוצא']
female_verbs = ['יודעת', 'מרגישה', 'צריכה', 'חושבת', 'מבינה', 'מצליחה', 'מאמינה', 'אוהבת', 'אומרת', 'לומדת', 'הולכת', 'מפחדת', 'מדברת', 'אמורה', 'מדריכה', 'שמחה', 'מתפללת', 'בטוחה', 'שואלת', 'נמצאת', 'חייבת', 'מכירה', 'משתדלת', 'מוצאת', 'שומרת', 'מצטערת', 'מבולבלת']

print(len(male_verbs))
print(len(female_verbs))


29
27


In [57]:
def count_tokens(tokens, specific_tokens):
    return len([tok for tok in tokens if tok in specific_tokens])

In [58]:
def male_to_female_verbs_ratio(tokens):
    male_verbs = ['יודע', 'מרגיש', 'חושב', 'מצליח', 'לומד', 'צריך', 'מדריך', 'מבין', 'אומר', 'אוהב', 'בטוח', 'מאמין', 'מדבר', 'שמח', 'מבקש', 'משתדל', 'שואל', 'חייב', 'מפחד', 'מתפלל', 'הולך', 'אמור', 'מכיר', 'מצטער', 'מתחיל', 'נופל', 'חזק', 'שומר', 'מוצא']
    female_verbs = ['יודעת', 'מרגישה', 'צריכה', 'חושבת', 'מבינה', 'מצליחה', 'מאמינה', 'אוהבת', 'אומרת', 'לומדת', 'הולכת', 'מפחדת', 'מדברת', 'אמורה', 'מדריכה', 'שמחה', 'מתפללת', 'בטוחה', 'שואלת', 'נמצאת', 'חייבת', 'מכירה', 'משתדלת', 'מוצאת', 'שומרת', 'מצטערת', 'מבולבלת']

    mr = count_tokens(tokens, male_verbs) # male rate
    fr = count_tokens(tokens, female_verbs) # female rate
    return  0 if (mr + fr) == 0 else (mr * fr) / (mr + fr)

In [59]:
data["Male_rate"] = data.apply(lambda row: count_tokens(row["Q_tokens_no_sw"], male_verbs), axis=1)

In [60]:
data["Female_rate"] = data.apply(lambda row: count_tokens(row["Q_tokens_no_sw"], female_verbs), axis=1)

In [61]:
data["Male_Female_ratio"] = data.apply(lambda row: male_to_female_verbs_ratio(row["Q_tokens_no_sw"]), axis=1)

In [62]:
data[["Male_rate", "Female_rate", "Male_Female_ratio"]].describe()

,Male_rate,Female_rate,Male_Female_ratio
count,5031.000000,5031.000000,5031.000000
mean,1.444246,1.594713,0.191094
std,2.600279,2.773871,0.514438
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,2.000000,2.000000,0.000000
max,41.000000,27.000000,9.283019


In [63]:
data[["Gender", "Male_rate", "Female_rate", "Male_Female_ratio"]].corr()

,Gender,Male_rate,Female_rate,Male_Female_ratio
Gender,1.000000,-0.345930,0.515743,0.223362
Male_rate,-0.345930,1.000000,-0.078367,0.288523
Female_rate,0.515743,-0.078367,1.000000,0.540702
Male_Female_ratio,0.223362,0.288523,0.540702,1.000000


In [64]:
feat_to_write = data[["Gender", "Question", "Answer", "Male_rate", "Female_rate"]]

In [65]:
feat_to_write

,Gender,Question,Answer,Male_rate,Female_rate
0,1,"\nשלום לכם, אני נער בן 14 וגר באחת ה""התנחלויות...","\nב""ה\nד´ שלום!\nראשית רציתי לומר לך שאני מאוד...",0,0
1,1,\nלעיתים יוצא לנו להתחיל את הסעודה בראש חודש ו...,"\nב""ה\nשלום וברכה\nהדבר נתון במחלוקת הפוסקים ה...",2,1
2,1,\nא ישר כח על האתר!\nאיך אפשר לנצל את החופש בצ...,"\nלק""י\n\nשלום לך \nראשית, תודה על המילים החמו...",0,0
3,1,"\nלמה בתפילה אומרים ""אלוקי אברהם, יצחק ויעקב"" ...","\nשלום רב לך! \nשאלה יפה שאלת, יש הבדל בין השמ...",0,0
4,1,\nהאם בחינוך צריך להקנות ערכים?\nיש ביסוס מהמק...,"\nמה שלומך שואל יקר?\nתמיד יפה ויותר מזה, חשוב...",1,0
...,...,...,...,...,...
5026,2,"\nשלום ,\nאני לא יודעת איך לכתוב את זה אני ינס...","\nשלום לך בחורה יקרה,\nאת מצטיירת בעיני כמישהי...",2,11
5027,2,\nבעזרתו יתברך!!! \nממש מצטערת שזו לא שאלה בנ...,"\nשלום לך,\nלא צריך להתנצל על לימוד תורה, ואפש...",0,2
5028,2,\nשלוםֱ!\nאני שומרת נגיעה יש לי ידידים והיתה ל...,"\nבס""ד\n\nשלום לך יקרה!!\n קודם כל, כל הכבוד ל...",0,8
5029,2,\nשלום! השאלה שלי היא כזו...\nיש אדם שגרם לי כ...,\nשלום וברכה!\nבאמת שאלה מסובכת.. אין לה תשובה...,0,8


Since the Male and Female Rates are based on tokens, that are the most frequent in the corpus, those features won't add significant value to the model, since they are already integrated in the features given by the count vector or the TF-IDF vector.
Hence, it is not Cost-effective to add those features to any model. moreover, in many lines both Male and Female Rate  are 0.

# testing & Playing

In [66]:
data

,Question,Answer,QuestionLen,AnswerLen,Gender,AnswerGender,Q_tokens_no_sw,A_tokens_no_sw,Q_bert_tokens_no_sw,A_bert_tokens_no_sw,Q_tokens_sw,Male_rate,Female_rate,Male_Female_ratio
0,"\nשלום לכם, אני נער בן 14 וגר באחת ה""התנחלויות...","\nב""ה\nד´ שלום!\nראשית רציתי לומר לך שאני מאוד...",182,1257,1,1.0,"[שלום, נער, בן, וגר, באחת, התנחלויות, אשמח, לד...","[שלום, ראשית, רציתי, לומר, לך, שאני, מבין, תחו...","[שלום, נער, בן, וגר, באחת, התנחלויות, אשמח, לד...","[שלום, ראשית, רציתי, לומר, לך, שאני, מבין, תחו...","[שלום, לכם, אני, נער, בן, וגר, באחת, התנחלויות...",0,0,0.000000
1,\nלעיתים יוצא לנו להתחיל את הסעודה בראש חודש ו...,"\nב""ה\nשלום וברכה\nהדבר נתון במחלוקת הפוסקים ה...",286,1095,1,1.0,"[לעיתים, יוצא, להתחיל, הסעודה, בראש, חודש, ולס...","[שלום, וברכה, הדבר, נתון, במחלוקת, הפוסקים, הא...","[לעיתים, יוצא, להתחיל, הסעודה, בראש, חודש, ולס...","[שלום, וברכה, הדבר, נתון, במחלוקת, הפוסקים, הא...","[לעיתים, יוצא, לנו, להתחיל, את, הסעודה, בראש, ...",2,1,0.666667
2,\nא ישר כח על האתר!\nאיך אפשר לנצל את החופש בצ...,"\nלק""י\n\nשלום לך \nראשית, תודה על המילים החמו...",198,767,1,1.0,"[ישר, כח, האתר, אפשר, לנצל, החופש, בצורה, נורמ...","[לק, שלום, לך, ראשית, תודה, המילים, החמות, באמ...","[ישר, כח, האתר, אפשר, לנצל, החופש, בצורה, נורמ...","[לק, שלום, לך, ראשית, תודה, המילים, החמות, באמ...","[ישר, כח, על, האתר, איך, אפשר, לנצל, את, החופש...",0,0,0.000000
3,"\nלמה בתפילה אומרים ""אלוקי אברהם, יצחק ויעקב"" ...","\nשלום רב לך! \nשאלה יפה שאלת, יש הבדל בין השמ...",77,741,1,NaN,"[בתפילה, אומרים, אלוקי, אברהם, יצחק, ויעקב, ול...","[שלום, רב, לך, שאלה, יפה, שאלת, הבדל, השמות, י...","[בתפילה, אומרים, אלוקי, אברהם, יצחק, ויעקב, ול...","[שלום, רב, לך, שאלה, יפה, שאלת, הבדל, השמות, י...","[למה, בתפילה, אומרים, אלוקי, אברהם, יצחק, ויעק...",0,0,0.000000
4,\nהאם בחינוך צריך להקנות ערכים?\nיש ביסוס מהמק...,"\nמה שלומך שואל יקר?\nתמיד יפה ויותר מזה, חשוב...",49,2529,1,1.0,"[האם, בחינוך, צריך, להקנות, ערכים, ביסוס, מהמק...","[שלומך, שואל, יקר, תמיד, יפה, ויותר, מזה, חשוב...","[האם, בחינוך, צריך, להקנות, ערכים, ביסוס, מהמק...","[שלומך, שואל, יקר, תמיד, יפה, ויותר, מזה, חשוב...","[האם, בחינוך, צריך, להקנות, ערכים, יש, ביסוס, ...",1,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5026,"\nשלום ,\nאני לא יודעת איך לכתוב את זה אני ינס...","\nשלום לך בחורה יקרה,\nאת מצטיירת בעיני כמישהי...",876,1662,2,2.0,"[שלום, יודעת, לכתוב, ינסה, הכי, טוב, שיש, חבר,...","[שלום, לך, בחורה, יקרה, מצטיירת, בעיני, כמישהי...","[שלום, יודעת, לכתוב, ינסה, הכי, טוב, שיש, חבר,...","[שלום, לך, בחורה, יקרה, מצט, ##יי, ##רת, בעיני...","[שלום, אני, לא, יודעת, איך, לכתוב, את, זה, אני...",2,11,1.692308
5027,\nבעזרתו יתברך!!! \nממש מצטערת שזו לא שאלה בנ...,"\nשלום לך,\nלא צריך להתנצל על לימוד תורה, ואפש...",213,1812,2,1.0,"[בעזרתו, יתברך, ממש, מצטערת, שזו, שאלה, בנושא,...","[שלום, לך, צריך, להתנצל, לימוד, תורה, ואפשר, ל...","[בעזרתו, יתברך, ממש, מצטערת, שזו, שאלה, בנושא,...","[שלום, לך, צריך, להתנצל, לימוד, תורה, ואפשר, ל...","[בעזרתו, יתברך, ממש, מצטערת, שזו, לא, שאלה, בנ...",0,2,0.000000
5028,\nשלוםֱ!\nאני שומרת נגיעה יש לי ידידים והיתה ל...,"\nבס""ד\n\nשלום לך יקרה!!\n קודם כל, כל הכבוד ל...",917,2530,2,2.0,"[שלוםֱ, שומרת, נגיעה, ידידים, והיתה, תקופה, שמ...","[בס, שלום, לך, יקרה, קודם, הכבוד, לך, ההסתכלות...","[שלום, שומרת, נגיעה, ידידים, והיתה, תקופה, שמד...","[בס, שלום, לך, יקרה, קודם, הכבוד, לך, ההסת, ##...","[שלוםֱ, אני, שומרת, נגיעה, יש, לי, ידידים, והי...",0,8,0.000000
5029,\nשלום! השאלה שלי היא כזו...\nיש אדם שגרם לי כ...,\nשלום וברכה!\nבאמת שאלה מסובכת.. אין לה תשובה...,807,1340,2,2.0,"[שלום, השאלה, כזו, אדם, שגרם, כאב, גדול, שהתמש...","[שלום, וברכה, באמת, שאלה, מסובכת, תשובה, ברורה...","[שלום, השאלה, כזו, אדם, שגרם, כאב, גדול, שהת, ...","[שלום, וברכה, באמת, שאלה, מסובכת, תשובה, ברורה...","[שלום, השאלה, שלי, היא, כזו, יש, אדם, שגרם, לי...",0,8,0.000000


In [67]:
texts = [" ".join(x) for x in data['Q_tokens_sw']]
print([i for i,x in enumerate(texts) if "בלויים אני" in x])
print([i for i,x in enumerate(texts) if "–אני" in x])
[i for i,x in enumerate(texts) if "s" in x]

[5]
[]


[]

In [68]:
texts[5]

'לכבוד הרב יש לי שאלה שמטרידה אותי כבר זמן רב אני בכור במשפחתי אני למדתי שבעיקרון הבכורים היו אמורים לשרת בבית המקדש אך בעקבות חטא העגל הם הוחלפו בלויים אני איני מבין את העונש הזה הרי חלקם של הבכורים היה בדיוק כמו של כל עם ישראל ולכן מדוע כל עם ישראל יצא נקי מכל העונש ורק קבוצה קטנה מהעם הבכורים נענשה אם התשובה היא שבני ישראל לא היו אמורים לקבל עונש בגלל תפילת משה ורק הלויים היו אמורים לקבל פרס כי הסכימו להרוג קרוביהם למען אז למה הם קיבלו את הפרס על חשבון הבכורים אשמח מאד לקבל תשובה במהרה'